In [64]:
import os
import re
import numpy as np
from fuzzywuzzy import fuzz
import pandas as pd
os.getcwd()


'/Volumes/GoogleDrive-118161624996893811890/My Drive/KGS/IT/Varun handover/KG_Somani/KG_Somani/test'

In [65]:
os.listdir()

['Legal_Apr and May_0089.xlsx',
 'HDFC 4748.xlsx',
 'pnb-4808.xlsx',
 'SVC_0017.xlsx',
 'BOM_8288_Statement_SEP.xlsx',
 'ICICI Bank_000705044202 (June).xlsx',
 '.ipynb_checkpoints',
 'test',
 'Merger-func.ipynb',
 'icici.xlsx',
 '.DS_Store',
 'icici_copy.xlsx',
 'test.ipynb',
 'ICICI_CR.xlsx',
 'test_k.ipynb',
 'Final_test_k.ipynb',
 'SBI.xlsx']

In [66]:
#reading dataframe and dropping those lines which have data in 2 col only
df=pd.read_excel('SBI.xlsx').dropna(axis=1, how='all')
df=df.dropna(thresh=len(df.columns)-2, axis=0)
df=df.apply( lambda x:x.astype(str).str.lower())
df.reset_index(inplace=True, drop=True)
#this bag contains the words which are  generally column headers for bank statement
bag=['transaction id','si. no.','tran id','txn no.','tran date','txn posted date','statement date','txn date','transaction date','post date','particulars','description','narrative','remarks','tran type','type','branch name','branch code','remitter branch','bank reference','chq','chequeno.','chq / ref number','cheque no.','cheque','utr number','reference','ref no./cheque no.','debit','dr amount','dr','withdrawl','credit','cr amount','cr','deposit','balance','available balance(inr)']   

In [67]:
#final_i is the index value of the row where we fidn the col relevant for dataframe 
final_i=0
ab=0
control=0
for i in df.index:
    x=0
    control=0
    for j in df.columns:
        tocheck=df.iloc[i,x]
        x+=1
        ab+=1
        if str(tocheck).lower() in bag:
            control+=1
            if control>3:
                final_i=i   
                break
    if control >3:
        break
print(type(final_i))

#deleting all the rows above the relevant header

if final_i==0:
    dfn=df
    print("yes")
else:
    print("no")
    dfn=df.drop(df.index[0:final_i])
    dfn.columns=dfn.iloc[0]
    dfn=dfn.drop(dfn.index[0])
    


<class 'int'>
yes


In [68]:
dfn.reset_index(inplace=True, drop=True)
col = dfn.columns
col = [i.lower() for i in col]
dfn.columns=col
dfn.columns
df=dfn
df.columns

Index(['txn date', 'value date', 'description', 'ref no./cheque no.',
       'branch code', '        debit', 'credit', 'balance'],
      dtype='object')

In [69]:
ideal_col=['Txn No.','Txn Date', 'Description',  'Bank', 'Account', 'Remarks',
               'Branch Name/Code', 'Cheque No.', 'Dr Amount', 'Cr Amount', 'Balance']

#transaction value/amount /amt -> +/- value // value Dr/Cr // Value and in next col Dr/Cr// Dr amount & Cr amount// Deposit & withdrawl

In [70]:
# check if the txn value is in the same column with CR DR in same column or diff column or txn value is negative is negative 
items = ['amount','value','amt','txn','transaction','Send','receive','debit','credit'] # probable words in txn value column
for i in range(len(df.columns)):
    colname=df.columns[i]
    
    ##incude a code which restrict this function to run further if there exist two columns -> one with dr/debit/withdraw/etc and another with cr/credit/deposit/etc
    control_c=set()
    for j in df.columns:
        if any(item in j for item in ['dr','debit','withdraw']):
            if not any(item in j for item in ['cr','credit','deposit']):
                ## this list contains all the name of 
                control_c.add(j)
            else:
                continue
        elif any(item in j for item in ['cr','credit','deposit']):
                 control_c.add(j)
        print("control_c = ",control_c)        
    if len(control_c)>1:
        break
    
    
    

    # to check if amount & value contains negative value or not (excludes dr amount/value or cr amount/value)
    if (any(item in colname for item in items)) or ('dr' in colname and 'cr' in colname): # help for task 1
        print(colname)
        if any(item in colname for item in ['dr','debit']):
            if any(item in colname for item in ['cr','credit']) in colname:
                pass # check next condition
            else:
                print('y3')
                continue ## next iteriation 
        elif any(item in colname for item in ['cr','credit']) in colname in colname:
            print('y4')
            continue ## next iteriation
        elif 'date' in colname:
            print('y5')
            continue ## next iteriation
        elif any(item in colname for item in ['withdraw','deposit']) in colname in colname:
            if 'withdraw' in colname:
                if 'deposit' in colname:
                    pass # next condition
                else:
                    break #Whether we should use break herefor next iteriation
            else:
                break ##Whether we should use break herefor next iteriation
            print('y6')
        else:
            pass
        print(colname, "yesss")
        
        pcolname=df.columns[i-1]
        ## check if there can be a next column or not
       
        x = df.columns
        Y= df.columns.tolist()
        print(Y)
        
        a=len(Y)
        print(a)
        
        if i>=len(df.columns):
            ncolname=df.columns[i+1]
            a=True
            print(ncolname)
        else:
            print("not found")
            a=False
       ##    ## check if there can be a next column or not TASK- 2  
    
        try:
            df[colname]=df[colname].astype(str).str.replace(',','', regex=True).astype(float)
        except:
            pass
#         print((df[df.columns[i-1]].str.contains('dr|cr')).any())
        
        try:
            if (df[colname].astype(str).str.contains('dr|cr')).any():
                print('2ndtry')
                df[colname]=df[colname].astype(str).str.replace(',','', regex=True)
                df['Dr amount'] = df[colname].apply(lambda x:x if 'dr' in x else 0)
                df['Cr amount'] = df[colname].apply(lambda x:x if 'cr' in x else 0) 
                df['Dr amount'] = df['Dr amount'].astype(str).str.replace('dr','', regex=True)
                df['Cr amount'] = df['Cr amount'].astype(str).str.replace('cr','', regex=True)
                df['Dr amount'] = df['Dr amount'].astype(str).str.replace(',','', regex=True).astype(float)
                df['Cr amount'] = df['Cr amount'].astype(str).str.replace(',','', regex=True).astype(float)
                
             
            elif ((df[colname]>0).all()):
                print('yaha')
                print((df[colname]>0).all())
                try:
                    print('b')
                            
                    if any(item in pcolname for item in ['description','remarks']): # if decription is in pcolname
                        print("TRUEEEEEEEEEEEEEEEEEEE")
                    else:
                         print("FALSEEEEEEEEEEEEEE")
                    
                    
                    ## TASK -3
                    if (df[pcolname].str.match('dr|cr')).any():
                        print('previous col has Dr|cr')
                        df['Dr amount'] = df.apply(lambda x:x[colname] if fuzz.ratio('dr|debit',x[pcolname])>80 else 0, axis =1)
                        df['Cr amount'] = df.apply(lambda x:x[colname] if fuzz.ratio('cr|credit',x[pcolname])>80 else 0, axis =1)
                        print("To check if code breaks here")
                        break # check if break is used appropriately here
                except:
                    pass
                
                try:
                    print('c')
                    # if there is no next col -> dont proceed further.
                    if (df[ncolname].str.match('dr|cr')).any() and a==True:  
                        df['Dr amount'] = df.apply(lambda x:x[colname] if fuzz.ratio('dr|debit',x[ncolname])>80 else 0, axis=1)
                        df['Cr amount'] = df.apply(lambda x:x[colname] if fuzz.ratio('cr|credit',x[ncolname])>80 else 0, axis=1)
                        break
                except:
                    print('d')
                    pass
            elif (df[colname]<0).any():
                df['Dr amount'] = df[colname].apply(lambda x:-x if x<0 else 0)
                df['Cr amount'] = df[colname].apply(lambda x:x if x>0 else 0)
                break
            else:
                print('na')
                
        except:
            print('naaa')        

control_c =  set()
control_c =  set()
control_c =  {'description'}
control_c =  {'description'}
control_c =  {'description'}
control_c =  {'description', '        debit'}
control_c =  {'credit', 'description', '        debit'}
control_c =  {'credit', 'description', '        debit'}


In [71]:
control_date=set()
for j in df.columns:
    if 'date' in j:
        control_date.add(j)
if len(control_date)>1:
    for j in control_date:
        if any(item in j for item in ['value','val','debit','credit']):
            print("yes")
            df.rename(columns = {j:'value date'}, inplace = True)
        elif any(item in j for item in ['transaction','txn','entry']):
            df.rename(columns = {j:'transaction date'}, inplace = True)
elif len(control_date)==1:
    for j in control_date:
        df.rename(columns = {j:'value date'}, inplace = True)    
else:
    df['value date']=''

yes


In [72]:
control_closingbalance=set()
for j in df.columns:
    if any(item in j for item in ['balance','available','bal','closing','opening','final',''])

,transaction date,value date,description,ref no./cheque no.,branch code,debit,credit,balance
0,2021-09-01,2021-09-01,to transfer-inb reversal for cr21336219--67000...,6700001040dce03701133 transfer t...,99922,13453440,,3295534.05
1,2021-09-01,2021-09-01,to transfer-corporate loan interest recovery a...,transfer to 39242339914 ...,18173,22420187,,-19124652.95
2,2021-09-01,2021-09-01,to transfer-wcl interest recovery aug 21-trans...,transfer to 40121788574 ...,18173,4001134,,-23125786.95
3,2021-09-01,2021-09-01,to transfer-sbi cmp charges for the month of a...,transfer to 31473896176 ...,4266,41,,-23125827.95
4,2021-09-01,2021-09-01,to transfer-inb interest charges for cr2142774...,5900083845ace03787045 transfer t...,99922,323840,,-23449667.95
...,...,...,...,...,...,...,...,...
360,2021-09-30,2021-09-30,to transfer-inb online neft-30092021t4cnabqflf...,30092021t4cnabqflfr0 transfer to...,99922,241820,,-62240929.59
361,2021-09-30,2021-09-30,to transfer-inb online neft-30092021t5cnabqfkt...,30092021t5cnabqfktj4 transfer to...,99922,15629092,,-77870021.59
362,2021-09-30,2021-09-30,transfer credit--sweep from 38937515222-,sweep from 38937515222 / -,18173,,9098897.34,-68771124.25
363,2021-09-30,2021-09-30,transfer credit--sweep from 35624497298-,sweep from 35624497298 / -,586,,26649428.25,-42121696.0
